# GR5293 - Proj1 - Group9
## Prediction on the shares of news
#### Machine learning pipeline with random forest
* Train test split
* K-fold cross validation
* Rank the feature importance
* Also make binned classifications?


#### Setup

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import matplotlib
from matplotlib import pyplot as plt
import time
import os
import re
import gc
gc.enable()
print(os.getcwd())
mydir = os.getcwd() + "/"

/Users/kangshuoli/Documents/VScode_workspace/GR5293/EODS-Project1-Group9/doc


#### Read in data

In [3]:
news_df = pd.read_csv(
    '../data/news_cleaned_ver2.csv', 
    header = 0
)
news_df = news_df.iloc[:,1:]
news_df.shape

(35103, 45)

#### Split the feature and response variable

In [4]:
feature_df = news_df.iloc[:,np.arange(news_df.shape[1] - 1)]
response_df = news_df.iloc[:,-1]
feature_df.shape

(35103, 44)

#### Show the predictors

In [5]:
feature_df.columns

Index(['n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'self_reference_min_shares',
       'self_reference_max_shares', 'self_reference_avg_sharess',
       'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday',
       'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday',
       'weekday_is_sunday', 'is_weekend', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'global_rate_negative_words', 'rate_positive_words',
       'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
       'max_positive_polarity', 'avg_negative_polarity',
       'min_negati

#### Modeling

In [6]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.model_selection import GridSearchCV

##### train test split

In [7]:
feature_train, feature_test, response_train, response_test = train_test_split(
    feature_df, 
    response_df, 
    test_size = 0.25, 
    shuffle = True, 
    random_state = 42
)
response_train = pd.DataFrame(response_train)
response_test = pd.DataFrame(response_test)
list_p1 = ["feature", "response"]
list_p2 = ["train", "test"]
for p1 in list_p1:
    for p2 in list_p2:
        temp_var = p1 + '_' + p2
        print(f'The Size of {temp_var} is: {globals()[temp_var].shape}')

# reshape the index
# make them start from 0 and end at their shape[0] - 1
for p1 in list_p1:
    for p2 in list_p2:
        temp_var = p1 + '_' + p2
        globals()[temp_var].index = np.arange(globals()[temp_var].shape[0], dtype = int)

The Size of feature_train is: (26327, 44)
The Size of feature_test is: (8776, 44)
The Size of response_train is: (26327, 1)
The Size of response_test is: (8776, 1)


##### feed data into the model

In [23]:
%xmode plain
# params_rf = {
#     'n_estimators': [10, 20, 50, 100, 200, 500, 1000, 2000], 
#     'criterion': ["squared_error", "poisson"], 
#     'max_depth': [None, 20, 15, 10, 8, 6, 4, 2], 
#     'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50], 
#     'min_smaples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40], 
#     'max_features': ["auto", "sqrt", "log2", 0.9, 0.8, 0.7, 0.6], 
#     'max_leaf_nodes': [None, 10, 20, 30, 40, 50], 
#     'min_impurity_decrease': [0.0, 0.1, 0.2, 0.5], 
#     'n_jobs': [-1], 
#     'bootstrap': [True], 
#     'oob_score': [True], 
#     'random_state': [42], 
#     'verbose': [0], 
#     'max_samples': [None, 0.9, 0.8, 0.7, 0.6]
# }

params_rf = {
    'n_estimators': [200, 500], 
    # 'criterion': ["squared_error"], 
    'max_depth': [None, 10, 5, 2], 
    # 'min_samples_split': [2, 5, 10], 
    # 'min_smaples_leaf': [1, 2, 5, 10], 
    'max_features': ["auto"], 
    'max_leaf_nodes': [None, 10, 20, 50], 
    # 'min_impurity_decrease': [0.0, 0.1, 0.2, 0.5], 
    'n_jobs': [-1], 
    'bootstrap': [True], 
    'oob_score': [True], 
    'random_state': [42], 
    'verbose': [0], 
    'max_samples': [None]
}


gridsearch = GridSearchCV(
    estimator = RandomForestRegressor(), 
    param_grid = params_rf, 
    scoring = "neg_root_mean_squared_error",
    cv = 10, 
    verbose = 1, 
    n_jobs = -1
)

gridsearch.fit(X = feature_train, y = np.array(response_train).ravel())
best_rf = gridsearch.best_estimator_
best_score = gridsearch.best_score_
best_params = gridsearch.best_params_


Exception reporting mode: Plain
Fitting 10 folds for each of 32 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:   

[CV] END bootstrap=True, max_depth=None, max_features=auto, max_leaf_nodes=None, max_samples=None, n_estimators=200, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time= 1.3min
[CV] END bootstrap=True, max_depth=None, max_features=auto, max_leaf_nodes=None, max_samples=None, n_estimators=200, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time= 1.3min
[CV] END bootstrap=True, max_depth=None, max_features=auto, max_leaf_nodes=None, max_samples=None, n_estimators=200, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time= 1.3min
[CV] END bootstrap=True, max_depth=None, max_features=auto, max_leaf_nodes=None, max_samples=None, n_estimators=200, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time= 1.3min
[CV] END bootstrap=True, max_depth=None, max_features=auto, max_leaf_nodes=None, max_samples=None, n_estimators=200, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time= 1.3min
[CV] END bootstrap=True, max_depth=None, max_

[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.2s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.2s finished
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done 180 task

[CV] END bootstrap=True, max_depth=None, max_features=auto, max_leaf_nodes=None, max_samples=None, n_estimators=200, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time= 1.3min


...

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 500 out of 500 | elapsed:    0.1s finished


[CV] END bootstrap=True, max_depth=2, max_features=auto, max_leaf_nodes=50, max_samples=None, n_estimators=500, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time=  28.9s


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 500 out of 500 | elapsed:    0.1s finished


[CV] END bootstrap=True, max_depth=2, max_features=auto, max_leaf_nodes=50, max_samples=None, n_estimators=500, n_jobs=-1, oob_score=True, random_state=42, verbose=1; total time=  21.8s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   17.0s finished


#### oof prediction and evaluation

In [28]:
# prediction
response_pred = best_rf.predict(feature_test)
pred_rmse = np.sqrt(mean_squared_error(response_pred, response_test))
print(f'The evaluation score for the best RandomForestRegressor is: {pred_rmse:0.4f}')

The evaluation score for the best RandomForestRegressor is: 1036.2315


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 500 out of 500 | elapsed:    0.1s finished


In [29]:
np.max(response_train)

shares    5500
dtype: int64

In [30]:
np.min(response_test)

shares    1
dtype: int64